# Component iteration (Py2)

In [ ]:
# Install Kubeflow Pipeline
!pip3 install kfp --upgrade

## Iterate on a Python function in a component

In [ ]:
import kfp.components as components
from kfp import compiler

def add(a: float, b: float) -> float:
    """This function docstring appears as the component description."""
    return a + b

GCP_PROJECT = '<YOUR PROJECT>'  # GCP Project under which container is created.
TAG = 'dev'                     # Tag for the created container.
IMAGE_PATH = 'gcr.io/%s/py2-iteration-component:%s' % (GCP_PROJECT, TAG)  # GCR path at which contianer is created.
GCS_PATH = 'gs://<YOUR GCS BUCKET>/'  # GCS Bucket used for staging.

add_op = compiler.build_python_component(
    component_func=add,
    staging_gcs_path=GCS_PATH,
    dependency=[
        # Additional PyPi dependencies.
        # If other dependency is necessary, %%docker should be used to crate the base image.
        kfp.compiler.VersionedDependency(name='google-api-python-client'),
    ],
    base_image='tensorflow/tensorflow:1.12.0',
    target_image=TARGET_IMAGE,
    python_version='python2')

## Build a pipeline with the component under iteration inside

In [ ]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='Lightweight pipeline',
   description='A toy pipeline that performs arithmetic calculations.'
)
def add_pipeline(
    a = '12',
    b = '15',
):
    add_task = add_op(a, b)

In [ ]:
pipeline_func = add_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.tar.gz'

import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [ ]:
from kfp_experiment.rest import ApiException

EXPERIMENT_NAME = 'Lightweight iteration'

#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()

try:
    experiment = client.create_experiment(EXPERIMENT_NAME)
except ApiException as e:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
    
run_name = pipeline_func.__name__ + ' Py2 Run'
run_result = client.run_pipeline(
    experiment.id, run_name, pipeline_filename, {'a': 10, 'b': 20})